<a href="https://colab.research.google.com/github/Amaan-Sama/CSRBOX_IBMSKILLBUILD/blob/main/Climate_Change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1

In [ ]:
!pip install -qU \
  google-generativeai==0.8.5 \
  google-ai-generativelanguage==0.6.15 \
  langgraph \
  langchain \
  langchain-google-genai \
  openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.5/757.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.6 MB/s eta 0:00:00


2

In [ ]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


3

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your gemini key")

Enter your gemini key··········


4

In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.2)

4

In [ ]:
def get_problem(state: dict) -> dict:
  problem = input("This is an Expert A.I guide for your problem , Please enter the situation to be solved")
  state["problem"] = problem
  return state

5

In [ ]:
def classify_problem(state : dict ) -> dict:
  prompt = (
      "You are a helpful environmental assistant. Classify the user's concern below into one of the categories:\n"
        "- Awareness\n- Action\n- Emergency\n\n"
        f"problem: {state['problem']}\n"
        "Respond with only one word: Awareness, Action, or Emergency.\n"
        "#Example: input: There is smoke in the sky from nearby fires. → output: Emergency"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM Classifies the problem as : {category}") #debug
  state["category"]=category

6


In [ ]:
def problem_router(state:dict) -> dict:
  cat = state["category"].lower()
  if "awareness" in cat:
    return"awareness"
  elif "emergency" in cat:
    return"emergency"
  elif "action" in cat:
    return "action"
  else:
    return "awareness"

7


In [ ]:
def awareness_node(state: dict) -> dict:
    prompt = (
        f"You are a knowledgeable climate educator.\n"
        f"User's question: {state['problem']}\n"
        "Give an informative, clear explanation to raise the user's awareness about this issue."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state

def emergency_node(state: dict) -> dict:
    prompt = (
        f"You are an environmental emergency assistant.\n"
        f"Situation: {state['problem']}\n"
        "Give urgent advice or immediate steps the user should take for their safety."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state

def action_node(state: dict) -> dict:
    prompt = (
        f"You are a helpful environmental assistant.\n"
        f"User's request: {state['problem']}\n"
        "Please provide a specific, actionable response to help the user take positive climate action based on their concern."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state

8

In [ ]:
builder = StateGraph(dict)


builder.set_entry_point("get_problem")


builder.add_node("get_problem", get_problem)
builder.add_node("classify", classify_problem)
builder.add_node("awareness", awareness_node)
builder.add_node("emergency", emergency_node)
builder.add_node("action", action_node)


builder.add_edge("get_problem", "classify")
builder.add_conditional_edges("classify", problem_router, {
    "awareness": "awareness",
    "emergency": "emergency",
    "action": "action"
})


builder.add_edge("awareness", END)
builder.add_edge("emergency", END)
builder.add_edge("action", END)


9


In [ ]:
graph = builder.compile()
final_state = graph.invoke({})
print("Final Output:")
print(final_state["answer"])

This is an Expert A.I guide for your problem , Please enter the situation to be solvedSaw a forest fire in Distance
LLM Classifies the problem as : Emergency
Final Output:
**DO NOT APPROACH THE FIRE.**  Your safety is the top priority.

**1. Immediately call emergency services:**  Your local fire department or emergency number (e.g., 911 in the US, 999 in the UK).  Give them the precise location of the fire (if possible using GPS coordinates or landmarks), the size of the fire (estimate), and the direction of the wind.

**2. Evacuate the area:** If you are close enough to be concerned about smoke inhalation or the fire spreading towards you, leave immediately.  Do not wait for official instructions if you feel threatened.  Take essential documents and medications with you.

**3. Stay informed:** Monitor news reports and official channels for updates on the fire and evacuation orders.

**4. Protect yourself from smoke:** If you cannot evacuate immediately, find a safe, sheltered locatio